**In this guide we will demonstrate how to use Nano AutoML to auto tune an machine learning model and handle the whole model development process easily.**

# Introduction
Nano provides hpo as an solution for fast and high-performance hyperparameter tuning. Users can import hano hpo following the standard tensorflow API without changing their own code. By setting the search space and running model.search(), the model will contain the best hyperparameters.

Nano hpo supports sequential, fuctional and custom keras API.


# Step0: Prepare Environment
We recommend using conda to prepare the environment.
conda create -n my_env python=3.7

conda activate my_env

pip install --pre --upgrade bigdl

# Step1: Init Nano AutoML
Let's start out by importing Nano AutoML and enable its tensorflow API globally by automl.hpo_config.enable_hpo_tf().

After importing, we will load MNIST as our data.

In [1]:
#to do: remove
import sys
sys.path.append("../../src/")

import bigdl.nano.automl as automl
automl.hpo_config.enable_hpo_tf()

from bigdl.nano.tf.keras.layers import Dense
from bigdl.nano.tf.keras import Input
from bigdl.nano.automl.tf.keras import Model
import bigdl.nano.automl.hpo.space as space


import tensorflow as tf
from tensorflow import keras

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

../../src/bigdl/nano/tf/__init__.py:24: UserWarning: NANO_TF_INTER_OP not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("NANO_TF_INTER_OP not found the in os.environ, "
../../src/bigdl/nano/tf/__init__.py:30: UserWarning: OMP_NUM_THREADS not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("OMP_NUM_THREADS not found the in os.environ, "


# Step2: Set search space
We now create our model function. 

Whenever you want to search a hyperparameter, you can simply replace it with a space and set your own searching space 


In [2]:
inputs = Input(shape=(784,))
x = Dense(units=space.Categorical(8,16,prefix='dense_1'), activation="relu")(inputs)
x = Dense(units=space.Categorical(32,64,prefix='dense_2'), activation="relu")(x)
outputs = Dense(units=10)(x)

model = Model(inputs=inputs, outputs=outputs, name="mnist_model")

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

2022-05-18 15:18:46.080250: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-18 15:18:46.081201: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


# Step3: Apply searching
Apply hyperparameter searching by calling model.search(), and best hyperparameters will be saved in the model.

You can specify the number of trial, each trial will use a set of hyperparameters automatiacally suggusted by nano.hpo. 

You can set your target metric and whehter you want to maximize you minimize it.

In [3]:
%%time
model.search(
    n_trials=5,
    target_metric='accuracy',
    direction="maximize",
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=24,
    validation_split=0.2,
    verbose=False,
)

[I 2022-05-18 15:18:46,242] A new study created in memory with name: no-name-48629828-cc12-4378-951f-4c113bc88e3c


Starting a new tuning
guoyao want to know compile args
() {'loss': <keras.losses.SparseCategoricalCrossentropy object at 0x7f7454564d50>, 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7f7454564510>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
375/375 - 2s - loss: 0.8201 - accuracy: 0.7711 - val_loss: 0.4023 - val_accuracy: 0.8854 - 2s/epoch - 4ms/step
Epoch 2/24
375/375 - 1s - loss: 0.3786 - accuracy: 0.8923 - val_loss: 0.3254 - val_accuracy: 0.9078 - 1s/epoch - 3ms/step
Epoch 3/24
375/375 - 1s - loss: 0.3233 - accuracy: 0.9083 - val_loss: 0.2873 - val_accuracy: 0.9180 - 1s/epoch - 3ms/step
Epoch 4/24
375/375 - 1s - loss: 0.2962 - accuracy: 0.9159 - val_loss: 0.2761 - val_accuracy: 0.9196 - 794ms/epoch - 2ms/step
Epoch 5/24
375/375 - 1s - loss: 0.2784 - accuracy: 0.9211 - val_loss: 0.2630 - val_accuracy: 0.9245 - 899ms/epoch - 2ms/step
Epoch 6/24
375/375 - 1s - loss: 0.2644 - accuracy: 0.9245 - val_loss: 0.2545 - val_accuracy: 0.9264 - 1s/epoch - 3ms/step
Epoch 7/24

[I 2022-05-18 15:19:12,689] Trial 0 finished with value: 0.9441041946411133 and parameters: {'units▁choice': 0}. Best is trial 0 with value: 0.9441041946411133.


guoyao want to know compile args
() {'loss': <keras.losses.SparseCategoricalCrossentropy object at 0x7f7434409c90>, 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7f74344095d0>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
375/375 - 2s - loss: 0.5834 - accuracy: 0.8439 - val_loss: 0.3073 - val_accuracy: 0.9107 - 2s/epoch - 4ms/step
Epoch 2/24
375/375 - 1s - loss: 0.3005 - accuracy: 0.9130 - val_loss: 0.2724 - val_accuracy: 0.9202 - 1s/epoch - 3ms/step
Epoch 3/24
375/375 - 2s - loss: 0.2542 - accuracy: 0.9262 - val_loss: 0.2230 - val_accuracy: 0.9337 - 2s/epoch - 5ms/step
Epoch 4/24
375/375 - 2s - loss: 0.2198 - accuracy: 0.9364 - val_loss: 0.2078 - val_accuracy: 0.9394 - 2s/epoch - 4ms/step
Epoch 5/24
375/375 - 1s - loss: 0.1954 - accuracy: 0.9427 - val_loss: 0.1879 - val_accuracy: 0.9434 - 1s/epoch - 3ms/step
Epoch 6/24
375/375 - 1s - loss: 0.1773 - accuracy: 0.9479 - val_loss: 0.1746 - val_accuracy: 0.9478 - 791ms/epoch - 2ms/step
Epoch 7/24
375/375 - 1s - loss: 0.1

[I 2022-05-18 15:19:40,049] Trial 1 finished with value: 0.9743750095367432 and parameters: {'units▁choice': 1}. Best is trial 1 with value: 0.9743750095367432.


...

guoyao want to know compile args
() {'loss': <keras.losses.SparseCategoricalCrossentropy object at 0x7f7434b8e9d0>, 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7f7434b8ea90>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
375/375 - 2s - loss: 0.8201 - accuracy: 0.7711 - val_loss: 0.4023 - val_accuracy: 0.8854 - 2s/epoch - 4ms/step
Epoch 2/24
375/375 - 1s - loss: 0.3786 - accuracy: 0.8923 - val_loss: 0.3254 - val_accuracy: 0.9078 - 1s/epoch - 4ms/step
Epoch 3/24
375/375 - 1s - loss: 0.3233 - accuracy: 0.9083 - val_loss: 0.2873 - val_accuracy: 0.9180 - 1s/epoch - 3ms/step
Epoch 4/24
375/375 - 1s - loss: 0.2962 - accuracy: 0.9159 - val_loss: 0.2761 - val_accuracy: 0.9196 - 1s/epoch - 3ms/step
Epoch 5/24
375/375 - 1s - loss: 0.2784 - accuracy: 0.9211 - val_loss: 0.2630 - val_accuracy: 0.9245 - 1s/epoch - 3ms/step
Epoch 6/24
375/375 - 1s - loss: 0.2644 - accuracy: 0.9245 - val_loss: 0.2545 - val_accuracy: 0.9264 - 1s/epoch - 4ms/step
Epoch 7/24
375/375 - 2s - loss: 0.2544

[I 2022-05-18 15:20:40,599] Trial 3 finished with value: 0.9441041946411133 and parameters: {'units▁choice': 0}. Best is trial 1 with value: 0.9743750095367432.


guoyao want to know compile args
() {'loss': <keras.losses.SparseCategoricalCrossentropy object at 0x7f743566d290>, 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7f743566d250>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
375/375 - 2s - loss: 0.8201 - accuracy: 0.7711 - val_loss: 0.4023 - val_accuracy: 0.8854 - 2s/epoch - 4ms/step
Epoch 2/24
375/375 - 1s - loss: 0.3786 - accuracy: 0.8923 - val_loss: 0.3254 - val_accuracy: 0.9078 - 1s/epoch - 3ms/step
Epoch 3/24
375/375 - 1s - loss: 0.3233 - accuracy: 0.9083 - val_loss: 0.2873 - val_accuracy: 0.9180 - 1s/epoch - 3ms/step
Epoch 4/24
375/375 - 2s - loss: 0.2962 - accuracy: 0.9159 - val_loss: 0.2761 - val_accuracy: 0.9196 - 2s/epoch - 5ms/step
Epoch 5/24
375/375 - 1s - loss: 0.2784 - accuracy: 0.9211 - val_loss: 0.2630 - val_accuracy: 0.9245 - 847ms/epoch - 2ms/step
Epoch 6/24
375/375 - 1s - loss: 0.2644 - accuracy: 0.9245 - val_loss: 0.2545 - val_accuracy: 0.9264 - 920ms/epoch - 2ms/step
Epoch 7/24
375/375 - 1s - loss: 

[I 2022-05-18 15:21:05,289] Trial 4 finished with value: 0.9441041946411133 and parameters: {'units▁choice': 0}. Best is trial 1 with value: 0.9743750095367432.


CPU times: user 8min 10s, sys: 9min 53s, total: 18min 4s
Wall time: 2min 19s


# Step 4: fit with the best hyperparameters
The best hyperparameters have been saved in the model. You can call model.fit() to train a model with the best hyperparameters.

In [4]:
history = model.fit(x_train, y_train,
                    batch_size=128, epochs=24, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

guoyao want to know compile args
() {'loss': <keras.losses.SparseCategoricalCrossentropy object at 0x7f7435e18590>, 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7f7435e18ad0>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
375/375 [==============================] - 2s 4ms/step - loss: 0.5834 - accuracy: 0.8439 - val_loss: 0.3073 - val_accuracy: 0.9107
Epoch 2/24
375/375 [==============================] - 1s 3ms/step - loss: 0.3005 - accuracy: 0.9130 - val_loss: 0.2724 - val_accuracy: 0.9202
Epoch 3/24
375/375 [==============================] - 1s 2ms/step - loss: 0.2542 - accuracy: 0.9262 - val_loss: 0.2230 - val_accuracy: 0.9337
Epoch 4/24
375/375 [==============================] - 1s 3ms/step - loss: 0.2198 - accuracy: 0.9364 - val_loss: 0.2078 - val_accuracy: 0.9394
Epoch 5/24
375/375 [==============================] - 2s 5ms/step - loss: 0.1954 - accuracy: 0.9427 - val_loss: 0.1879 - val_accuracy: 0.9434
Epoch 6/24
375/375 [==============================] - 2s 4ms

Check out the summary of the model. The model has already been built with the best hyperparameters found by nano hpo.

In [5]:
print(model.summary())

Model: "mnist_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_3 (Dense)             (None, 16)                12560     
                                                                 
 dense_4 (Dense)             (None, 64)                1088      
                                                                 
 dense_5 (Dense)             (None, 10)                650       
                                                                 
Total params: 14,298
Trainable params: 14,298
Non-trainable params: 0
_________________________________________________________________
None
